In [93]:
!pip install litellm
!pip install termcolor

Reshimming asdf python...
Reshimming asdf python...


In [102]:
from litellm import completion

messages = [
    {"content": "Hello, how are you?","role": "user"},
]

response = completion(
    model="ollama/phi3:14b",
    messages=messages,
    api_base="http://localhost:11434"
)

In [103]:
response.choices[0].message.content

"I'm doing well! As an artificial intelligence, I don't have feelings or emotions, but I'm here and ready to assist you. How can I help you today?"

In [104]:
code_str = """
from dataclasses import dataclass

@dataclass
class Item:
    \"\"\"Class for keeping track of an item in inventory.\"\"\"
    quantity: int = 0

Item.quantity = 41
result = Item.quantity + 1
"""
# eval("41 + 1")
local_scope = {}
exec(code_str, {}, local_scope)
print(local_scope)
print(local_scope["result"])

{'dataclass': <function dataclass at 0x107c69af0>, 'Item': <class 'Item'>, 'result': 42}
42


In [105]:
from dataclasses import dataclass, field
from datetime import datetime
from typing import ClassVar

@dataclass
class Event:
    timestamp: datetime = field(default_factory=datetime.now)

    @property
    def message(self) -> str:
        return ''

@dataclass
class Action(Event):
    runnable: ClassVar[bool] = False

@dataclass
class CmdRunAction(Action):
    command: str = ''
    thought: str = ''
    runnable: ClassVar[bool] = True

    @property
    def message(self) -> str:
        return f'Running command: {self.command}'

    def __str__(self) -> str:
        ret = '**CmdRunAction**\n'
        if self.thought:
            ret += f'THOUGHT: {self.thought}\n'
        ret += f'COMMAND: {self.command}'
        return ret

@dataclass
class Observation(Event):
    content: str = ''

@dataclass
class CmdOutputObservation(Observation):
    command_id: int = 0
    command: str = ''
    exit_code: int = 0

    @property
    def error(self) -> bool:
        return self.exit_code != 0

    @property
    def message(self) -> str:
        return f'Command `{self.command}` executed with exit code {self.exit_code}.'

    def __str__(self) -> str:
        return f'**CmdOutputObservation (exit code={self.exit_code})**\n{self.content}'
        
cmd_action = CmdRunAction(command="ls -l", thought="Listing directory contents")
print(cmd_action.message)
print(cmd_action)

Running command: ls -l
**CmdRunAction**
THOUGHT: Listing directory contents
COMMAND: ls -l


In [106]:
import subprocess

def run_bash(command: str) -> str:
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        return result.stdout
    else:
        return f'Error: {result.stderr}'

run_bash(cmd_action.command)

'total 80\n-rw-r--r--@ 1 dimova01  staff  40127 24 May 16:44 github_insights.ipynb\n'

In [107]:
import re

SYSTEM_PROMPT = """
You are an AI Software Engineer designed to help the user complete their task. As an expert in bash, you primarily achieve your tasks using bash commands and scripts.
To assist the user, you have the ability to execute bash commands on their behalf.
When you want to run a command, use the following format in your response:

<execute_bash>COMMAND_HERE</execute_bash>

Replace COMMAND_HERE with the actual bash command you want to execute.
After providing the command, you will receive the result of the execution, which you can then use to formulate your next response to the user.

Your goal is to engage in a conversation with the user, understand their requirements, and use bash commands as needed to fulfill their request.
Remember to break down complex tasks into smaller steps and execute commands one at a time to keep the process simple and easy to follow.

Always prioritize providing clear and concise responses to the user, and only use bash commands when absolutely necessary to complete the task at hand.
""".strip()

def parse_llm_response(response: str) -> str:
    match = re.search(r"<execute_bash>(.*?)</execute_bash>", response, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return None

def call_llm(messages: list) -> str:
    response = completion(
        model="ollama/phi3:14b",
        messages=messages,
        tempreture=0.3,
        api_base="http://localhost:11434"
    )
    return response

In [113]:
project_path = "/Users/dimova01/code/NeMo-Guardrails"
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": f"""
    Please locate and read the README.md file in the project located at {project_path}.
    Once you have read the file, provide a concise summary of what the project is about, its main features, and its goals.
    If the README.md file is missing or cannot be found, please let me know.
""".strip()},
]
print(colored(messages, "red"))

[{'role': 'system', 'content': 'You are an AI Software Engineer designed to help the user complete their task. As an expert in bash, you primarily achieve your tasks using bash commands and scripts.\nTo assist the user, you have the ability to execute bash commands on their behalf.\nWhen you want to run a command, use the following format in your response:\n\n<execute_bash>COMMAND_HERE</execute_bash>\n\nReplace COMMAND_HERE with the actual bash command you want to execute.\nAfter providing the command, you will receive the result of the execution, which you can then use to formulate your next response to the user.\n\nYour goal is to engage in a conversation with the user, understand their requirements, and use bash commands as needed to fulfill their request.\nRemember to break down complex tasks into smaller steps and execute commands one at a time to keep the process simple and easy to follow.\n\nAlways prioritize providing clear and concise responses to the user, and only use bash c

In [114]:
resp = call_llm(messages)
print(resp)
response = resp.choices[0].message.content

ModelResponse(id='chatcmpl-c37dae05-2796-4165-9469-e20fe8268e60', choices=[Choices(finish_reason='stop', index=0, message=Message(content='To locate and read the README.md file at /Users/dimova01/code/NeMo-Guardrails, we can use the `cat` command:\n\n<execute_bash>cd /Users/dimova01/code/NeMo-Guardrails && cat README.md</execute_bash>\n\nI will now execute this command and wait for the result to provide a summary of the project based on its contents.', role='assistant'))], created=1716565785, model='ollama/phi3:14b', object='chat.completion', system_fingerprint=None, usage=Usage(prompt_tokens=4, completion_tokens=100, total_tokens=104))


In [115]:
response
print(response)
parsed_response = parse_llm_response(response)

To locate and read the README.md file at /Users/dimova01/code/NeMo-Guardrails, we can use the `cat` command:

<execute_bash>cd /Users/dimova01/code/NeMo-Guardrails && cat README.md</execute_bash>

I will now execute this command and wait for the result to provide a summary of the project based on its contents.


In [116]:
from termcolor import colored

cmd_output = run_bash(parsed_response)
print(colored(f"Output of the command:\n{cmd_output}", "green"))

messages.append({"role": "user", "content": f"I located and read the `README.md` file. Here's a summary of its contents:\n\n{cmd_output}"})

resp = call_llm(messages)
print(colored(f"Assistant: {resp.choices[0].message.content}", "blue"))

Output of the command:
# NeMo Guardrails

[![Tests](https://img.shields.io/badge/Tests-passing-green)](#)
[![License](https://img.shields.io/badge/License-Apache%202.0-brightgreen.svg)](https://github.com/NVIDIA/NeMo-Guardrails/blob/main/LICENSE.md)
[![Project Status](https://img.shields.io/badge/Status-beta-orange)](#)
[![PyPI version](https://badge.fury.io/py/nemoguardrails.svg)](https://badge.fury.io/py/nemoguardrails)
[![Python 3.7+](https://img.shields.io/badge/python-3.7%2B-green)](https://www.python.org/downloads/)
[![Code style: black](https://img.shields.io/badge/code%20style-black-000000.svg)](https://github.com/psf/black)
[![arXiv](https://img.shields.io/badge/arXiv-2310.10501-b31b1b.svg)](https://arxiv.org/abs/2310.10501)

> **LATEST RELEASE / DEVELOPMENT VERSION**: The [main](https://github.com/NVIDIA/NeMo-Guardrails/tree/main) branch tracks the latest released beta version: [0.7.1](https://github.com/NVIDIA/NeMo-Guardrails/tree/v0.7.1). For the latest development version,